In [2]:
import requests
import pandas as pd
import numpy as np
import config


In [3]:
urls = {'profit_loss':'https://mops.twse.com.tw/mops/web/ajax_t163sb04', #綜合損益
        'asset_debt': 'https://mops.twse.com.tw/mops/web/t163sb05', #資產負債,
        'operation_revenue': 'https://mops.twse.com.tw/mops/web/t21sc04_ifrs', #營業收入
        'operation_analysis': 'https://mops.twse.com.tw/mops/web/t163sb06'#, 營業分析
        #'gross': 'https://mops.twse.com.tw/mops/web/t163sb07' #毛利率(可以從綜合損益找到）
        }



In [4]:


def query(year, data_type):
    dfs = []
    for i in range(4):
        r = requests.post(urls[data_type], {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        #'co_id':stock_id, 
        'year':str(year),
        'season':str(i+1),
        })
        r.encoding = 'utf8'
        dfs.append(pd.read_html(r.text, header=None))
    return dfs

def extration_by_season_per_year(df_list, column_names, company_type, year):
    #company_type = 3
    norms = {}
    for col in column_names[2:]:
        norm_data = df_list[0][company_type][['公司代號', '公司名稱']]
        for i in range(4):
            #print(i)
            new_data = df_list[i][company_type][['公司代號', '公司名稱', col]]
            new_data.columns = ['公司代號', '公司名稱', f"{year}_{i+1}"]
            norm_data = norm_data.merge(new_data, how='outer')
        norms[col] = norm_data
    return norms






In [5]:
data = query(109, 'profit_loss')

# data[i][j]
# i: season
# j:
#    1: bank
#    3: standard
#    4: holdings
#    5: insurance
#    6: other 

In [28]:
norm = extration_by_season_per_year(data, config.profit_loss_col['standard'], 3, 109)

In [29]:
norm

{'營業收入':      公司代號    公司名稱       109_1       109_2       109_3      109_4
 0    1101      台泥  21976200.0  52439827.0  82248488.0  114367247
 1    1102      亞泥  13138882.0  33828093.0  54564173.0   78240880
 2    1103      嘉泥    495334.0    992319.0   1481489.0    2058417
 3    1104      環泥   1253758.0   2591631.0   3886551.0    5426217
 4    1108      幸福   1104720.0   2226696.0   3358346.0    4530172
 ..    ...     ...         ...         ...         ...        ...
 906  6515      穎崴         NaN   1460781.0   2287484.0    2909821
 907  6743     安普新         NaN   1336698.0   3413171.0    4998255
 908  6776    展碁國際         NaN   8072571.0  12879557.0   17281341
 909  6768   志強-KY         NaN         NaN         NaN   13514535
 910  6781  AES-KY         NaN         NaN         NaN    6306568
 
 [911 rows x 6 columns],
 '營業成本':      公司代號    公司名稱       109_1       109_2       109_3     109_4
 0    1101      台泥  16195489.0  35779548.0  55034502.0  76779667
 1    1102      亞泥  10440940.0  244